In [1]:
# Our experimental complexities need to run over a network (Product Space)
# How to build a Product Space is far from trivial
# The following is inspired by the original Product Space of the Atlas but uses methods statistically more sound
import sys
import pandas as pd
import networkx as nx
sys.path.append("../..")
from libraries import backboning as bb

In [6]:
# The Product Space is based on the MCP_exp matrix
# We binarize it to be consistent with how we calculate complexities
# We start with RCA then later we'll do RPOP as well
mcp_exp = pd.read_csv("../../data/processed/mcp_exp.csv", sep = "\t", dtype = {"prod": str})
mcp_rca = pd.pivot_table(data = mcp_exp, index = "prod", columns = "exporter", values = "rca").fillna(0.0) >= 1
mcp_rca

exporter,ABW,AFG,AGO,ALB,ARE,ARG,ARM,AUS,AUT,AZE,...,URY,USA,UZB,VEN,VGB,VNM,YEM,ZAF,ZMB,ZWE
prod,,,,,,,,,,,,,,,,,,,,,
0101,False,False,False,False,True,True,False,True,False,False,...,True,True,False,False,False,False,False,False,False,False
0102,False,True,False,False,False,False,False,True,True,False,...,True,False,False,False,False,False,False,False,False,False
0103,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0104,False,False,False,False,False,False,True,True,False,False,...,True,False,False,False,False,False,False,False,False,False
0105,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9618,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
9701,False,False,False,False,False,False,False,False,True,False,...,False,True,False,False,True,False,False,False,False,False
9703,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,True


In [7]:
# My backboning library requires a pandas dataframe that is not triangular, even if the network is undirected
# The adjacency matrix should be weighted with the count of common exporter for each products
# This is easily achieved via dot product
ps = mcp_rca.astype(int).dot(mcp_rca.astype(int).T)
ps = ps.unstack()
# The library wants the columns to be called src, trg, and nij (the latter is the weight column)
ps.index.names = ("src", "trg")
ps = ps.reset_index().rename(columns = {0: "nij"})
# Throw away self loops
ps = ps[ps["src"] != ps["trg"]]
# Throw away empty links (products pairs without co-exporters)
ps = ps[ps["nij"] > 0]
ps

,src,trg,nij
1,0101,0102,9
2,0101,0103,5
3,0101,0104,5
4,0101,0105,7
5,0101,0106,9
...,...,...,...
988029,9706,9616,1
988031,9706,9618,4
988032,9706,9701,8
988033,9706,9703,8


In [8]:
# Calculate the backbone scores for each link using the noise corrected method
# Method described in https://arxiv.org/abs/1701.07336
ps_nc = bb.noise_corrected(ps, undirected = True, calculate_p_value = True)

# We need to find a reasonable threshold to say that an edge is statistically significant
# This is tricky because we also want every product to have at least a link
# and we also want the Product Space to be a single connected component

# First we start by saying that we only want links with a p-value < 0.01
ps_nc_bb = bb.thresholding(ps_nc, 0.99)
ps_nc_bb

Calculating NC score...


,src,trg,nij,score
997,0102,0104,20,0.999976
2987,0104,0106,17,0.999319
6,0101,0201,13,0.999484
1000,0102,0201,20,0.998815
7,0101,0202,7,0.993049
...,...,...,...,...
905468,8901,9013,3,0.994506
913420,9001,9013,3,0.996665
914414,9002,9013,3,0.997031
920378,9010,9013,2,0.995481


In [13]:
# Now we need to keep track of all products that don't have a link and the number of connected components in the Product Space
products = set(mcp_exp["prod"])
G = nx.from_pandas_edgelist(ps_nc_bb, source = "src", target = "trg", edge_attr = ("nij", "score"))
missing_products = products - set(G.nodes)
G_comps = list(nx.connected_components(G))
print("Products without a link:")
print(missing_products)
print(f"Connected components: {len(G_comps)}")

Products without a link:
{'3209', '8476', '3918', '9406', '3214', '5601', '7325', '1518', '8212', '2403', '6905', '2833', '3405', '7412', '4006', '7604', '6914', '7606', '0408', '6810', '3210', '8405', '7413', '3005', '8478', '8546', '3917', '4010', '7019', '5107', '2104', '4901', '8309', '7010', '2309', '2206', '8426', '3814', '6809', '2105', '8436', '4808', '3505', '4902', '3402', '3208', '4012', '8404', '7310', '1806', '8449', '1702', '5404', '7308', '8417', '4821', '2839', '4818', '8425', '2522', '3923', '9602', '5907', '3816', '6902', '5602', '7311', '4805', '9304', '3909', '7612', '7307', '7608', '2904', '3920', '4807', '7005', '7324', '7609', '8703', '4817', '3307', '4811', '3406', '3305', '3809', '3926', '4008', '3921', '2003'}
Connected components: 3


In [14]:
# We keep adding edges until we have a single connected components and no product has zero links
while len(G_comps) > 1 or len(missing_products) > 0:
    # First we need to know in which component each node is
    node2comp = {}
    for node in products:
        for i, c in enumerate(G_comps):
            _ = [i for i, c in enumerate(G_comps) if node in c]
            if len(_) == 0:
                # Isolated nodes are in a component identifid by their own code
                node2comp[node] = node
            else:
                # Nodes in a component get the component id
                node2comp[node] = _[0]
    # Assign component id to the nodes
    ps_nc["comp_src"] = ps_nc["src"].map(node2comp)
    ps_nc["comp_trg"] = ps_nc["trg"].map(node2comp)
    # We pick the most significant edge that connects two nodes in different components
    # This unifies the two components and/or reduces the number of isolated nodes
    new_edge = ps_nc[ps_nc["comp_src"] != ps_nc["comp_trg"]].sort_values(by = "score").iloc[-1]
    # Add the edge to the Product Space
    G.add_edge(new_edge["src"], new_edge["trg"], nij = new_edge["nij"], score = new_edge["score"])
    # Update the stopping conditions
    G_comps = list(nx.connected_components(G))
    missing_products = products - set(G.nodes)
    print(f"Connected components: {len(G_comps)}, Missing product count: {len(missing_products)}")

Connected components: 3, Missing product count: 89
Connected components: 3, Missing product count: 88
Connected components: 3, Missing product count: 87
Connected components: 3, Missing product count: 86
Connected components: 3, Missing product count: 85
Connected components: 3, Missing product count: 84
Connected components: 3, Missing product count: 83
Connected components: 3, Missing product count: 82
Connected components: 3, Missing product count: 81
Connected components: 3, Missing product count: 80
Connected components: 3, Missing product count: 79
Connected components: 3, Missing product count: 78
Connected components: 3, Missing product count: 77
Connected components: 3, Missing product count: 76
Connected components: 3, Missing product count: 75
Connected components: 3, Missing product count: 74
Connected components: 3, Missing product count: 73
Connected components: 3, Missing product count: 72
Connected components: 3, Missing product count: 71
Connected components: 4, Missin

In [15]:
# Now we have a Product Space with p < 0.01 edges, except the ones tying it together in a single component
# This is based on RCA, so let's name it accordingly
nx.write_edgelist(G, "ps_nc_bb_0.01_rca.csv", delimiter = "\t", data = ["nij", "score"])

In [17]:
# And now we do it all again, but for RPOP instead of RCA
# Maybe this should be a utility function
mcp_rpop = pd.pivot_table(data = mcp_exp, index = "prod", columns = "exporter", values = "rpop").fillna(0.0) >= 0.25

ps = mcp_rpop.astype(int).dot(mcp_rpop.astype(int).T)
ps = ps.unstack()
ps.index.names = ("src", "trg")
ps = ps.reset_index().rename(columns = {0: "nij"})
ps = ps[ps["src"] != ps["trg"]]
ps = ps[ps["nij"] > 0]

ps_nc = bb.noise_corrected(ps, undirected = True, calculate_p_value = True)

# We start with a much lower significance threshold (0.9 instead of 0.99)
# Because RPOP edges seem to be much more noisy...
ps_nc_bb = bb.thresholding(ps_nc, 0.9)

products = set(mcp_exp["prod"])
G = nx.from_pandas_edgelist(ps_nc_bb, source = "src", target = "trg", edge_attr = ("nij", "score"))
missing_products = products - set(G.nodes)
G_comps = list(nx.connected_components(G))

while len(G_comps) > 1 or len(missing_products) > 0:
    node2comp = {}
    for node in products:
        for i, c in enumerate(G_comps):
            _ = [i for i, c in enumerate(G_comps) if node in c]
            if len(_) == 0:
                node2comp[node] = node
            else:
                node2comp[node] = _[0]
    ps_nc["comp_src"] = ps_nc["src"].map(node2comp)
    ps_nc["comp_trg"] = ps_nc["trg"].map(node2comp)
    new_edge = ps_nc[ps_nc["comp_src"] != ps_nc["comp_trg"]].sort_values(by = "score").iloc[-1]
    G.add_edge(new_edge["src"], new_edge["trg"], nij = new_edge["nij"], score = new_edge["score"])
    G_comps = list(nx.connected_components(G))
    missing_products = products - set(G.nodes)
    print(f"Connected components: {len(G_comps)}, Missing product count: {len(missing_products)}")

nx.write_edgelist(G, "ps_nc_bb_0.1_rpop.csv", delimiter = "\t", data = ["nij", "score"])

Calculating NC score...


Connected components: 7, Missing product count: 352
Connected components: 7, Missing product count: 351
Connected components: 7, Missing product count: 350
Connected components: 7, Missing product count: 349
Connected components: 7, Missing product count: 348
Connected components: 7, Missing product count: 347
Connected components: 7, Missing product count: 346
Connected components: 8, Missing product count: 344
Connected components: 7, Missing product count: 344
Connected components: 7, Missing product count: 343
Connected components: 7, Missing product count: 342
Connected components: 7, Missing product count: 341
Connected components: 7, Missing product count: 340
Connected components: 7, Missing product count: 339
Connected components: 7, Missing product count: 338
Connected components: 7, Missing product count: 337
Connected components: 7, Missing product count: 336
Connected components: 7, Missing product count: 335
Connected components: 7, Missing product count: 334
Connected co